In [0]:
import pandas as pd

# dbutils.fs.rm('/FileStore/tables/', recurse=True)
# dbutils.fs.mkdirs('dbfs:/FileStore/tables')
files = dbutils.fs.ls('/FileStore/Bin_Estimation')
display(files)

# dbutils.fs.mkdirs('dbfs:/FileStore/Bin_Estimation/tables')
# dbutils.fs.mv('dbfs:/FileStore/tables', 'dbfs:/FileStore/Bin_Estimation/tables', recurse=True)

# display(files)
# l = [f.name for f in files]
# print('num files: ', len(l))
# display(pd.DataFrame(l, columns=['installations']))

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas


spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

#filtered_df = df.filter(df.installation_id is unique)
# display(df.select("installation_id").distinct())
# display(df.select("tag").distinct())

df = spark.sql("""
               select * from dev_salesforce.bronze.sf_site
            """)
display(df)

# df = spark.table('test_unify_analytics.bronze.logs_partition_1') \
#         .select('installation_id', 'tag', 'unix_timestamp', 'values') \
#         .filter((col("tag") == 242) & (col('values')[5] == 14))
# df = spark.sql("""
#                 select installation_id, tag, unix_timestamp, values
#                 from test_unify_analytics.bronze.logs_partition_1
#                 where tag = 242 AND values[5] = 14
#                 """)
# display(df)




In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

# H-E-B Plano
df = spark.sql("""
               select installation_id, unix_timestamp, tag, values
               from test_unify_analytics.bronze.logs_partition_1
               where installation_id = 'a031v00001F9MCKAA3'
            """)

# df_bu_content = df.filter((df['tag'] == 242) & (df['values'][6].isNotNull())).groupBy('bin_num').count().sort('bin_num')
# df_bc_content = df.filter((df['tag'] == 362) & (df['values'][1].isNotNull())).groupBy('bin_num').count().sort('bin_num')
# df_merge = df_bu_content.join(df_bc_content, on='bin_num', how='outer').sort('bin_num')
# print('bin_update content:', df_bu_content.select(sum('count')).collect()[0][0])
# print('bin_content:', df_bc_content.select(sum('count')).collect()[0][0])
# display(df_bu_content)
# display(df_bc_content)
# display(df_merge)

isBIN_UPDATE = df['tag'] == 242
isPORT_CLOSEBIN = df['tag'] == 249
# df_bu_content = df.filter((df['tag'] == 242) & (df['values'][6].isNotNull())) \
#                   .withColumn('bin_num', df['values'][0]) \
#                   .withColumn('content_code', df['values'][6]) \
#                   .orderBy(['bin_num', 'local_installation_timestamp'], ascending=True)
# df_bc_content = df.filter((df['tag'] == 362) & (df['values'][1].isNotNull())) \
#                   .withColumn('bin_num', df['values'][0]) \
#                   .withColumn('content_code', df['values'][1]) \
#                   .orderBy(['bin_num', 'local_installation_timestamp'], ascending=True)

df_cb_content = df.filter((isPORT_CLOSEBIN) & (df['values'][3].isNotNull())) \
                  .withColumn('bin_num', df['values'][1]) \
                  .withColumn('content_code', df['values'][3]) \
                  .groupBy(['installation_id', 'content_code']).count() \
                  .orderBy('count', ascending=False)

df_bu_depth = df.filter((isBIN_UPDATE) & (df['values'][4].isNotNull())) \
                  .withColumn('bin_num', df['values'][0]) \
                  .withColumn('content_code', df['values'][6]) \
                  .withColumn('depth', df['values'][4]) \
                  .orderBy(['unix_timestamp'], ascending=True)

display(df_cb_content)




In [0]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

data = tuple((row['content_code'], row['count']) for row in df_cb_content.collect())
print(len(data))
if len(data) == 0:
    dbutils.notebook.exit("no data")
    
content_code, count = zip(*data)
# print(type(data[0][0]))
print('content codes: ', content_code)
print('counts: ', count)

kmeans = KMeans(n_clusters=2)
kmeans.fit(data)
# print(kmeans.labels_)
centers = kmeans.cluster_centers_
grouping = kmeans.labels_
print('grouping 1: ', grouping)
print(type(grouping))

if grouping[0] == 0:
    grouping = grouping ^ 1
    
print('grouping 2: ', grouping)

content_code_associated_groups = zip(content_code, grouping)
content_code_associated_groups = sorted(content_code_associated_groups, key=lambda x: x[1], reverse=True)

print(content_code_associated_groups)


In [0]:
display(df_bu_depth.drop('installation_id', 'tag').sort(['bin_num', 'unix_timestamp']))


In [0]:
fig, axs = plt.subplots(1, 2)

axs[0].scatter(content_code, count, c=kmeans.labels_, s=10)
axs[0].scatter(centers[:, 0], centers[:, 1], c='red', s=20, alpha=0.5, marker='X')
axs[0].set_yscale('log')
axs[0].set_xlabel('content code')
axs[0].set_ylabel('count')
axs[0].set_ylim(0, 10e8)
axs[0].grid(True, which='both', linestyle='--', linewidth=0.5)

axs[1].scatter(centers[:, 0], centers[:, 1], c='red', s=20, alpha=0.5, marker='X')
axs[1].set_xlabel('content code')
axs[1].set_yscale('log')
axs[1].set_ylim(0, 10e8)
axs[1].grid(True, which='both', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

In [0]:
list = [0, 1, 1, 0]

print([1 - x for x in list])